In [1]:
# Setup notebook
import random
import numpy as np
import evaluation.plot as plot

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules every time a cell is executed
%autoreload 2

# Call the function to customize the default template
plot.customize_default_template()

seed = 0
random.seed(seed)
np.random.seed(seed)

In [2]:
from dataset.common import resolve_folds
from dataset.movie_lens import load_ml_100k_folds

data, k_fold = load_ml_100k_folds()
folds = resolve_folds(data, k_fold)

folds_without_index = [fold[1] for fold in folds]

Already downloaded!. Nothing to do.


In [3]:
from evaluation.strategies import (
    MAEStrategy,
    RMSEStrategy,
    MicroAveragedRecallStrategy,
    MacroAveragedRecallStrategy,
    RecallAtKStrategy,
    MicroAveragedPrecisionStrategy,
    MacroAveragedPrecisionStrategy,
    PrecisionAtKStrategy,
    NDCGStrategy,
    F1ScoreStrategy,
    PredictionCoverageStrategy,
)

train_measures = [ ]

test_measures = [
    MAEStrategy(verbose=False),
    RMSEStrategy(verbose=False),
    MicroAveragedRecallStrategy(threshold=4.0),
    MacroAveragedRecallStrategy(threshold=4.0),
    RecallAtKStrategy(k=20, threshold=4.0),
    MicroAveragedPrecisionStrategy(threshold=4.0),
    MacroAveragedPrecisionStrategy(threshold=4.0),
    PrecisionAtKStrategy(k=20, threshold=4.0),
    NDCGStrategy(k=20, threshold=4.0),
    F1ScoreStrategy(k=20, threshold=4.0),
    PredictionCoverageStrategy(),
]


In [4]:
from surprise.prediction_algorithms import KNNWithMeans
from evaluation.benchmark import GridSearch

parameters_grid = {
    "k": [1, 5, 10, 20, 30, 50, 100, 200],
    "min_k": [1, 10, 20, 30],
    "sim_options": [
        {"name": "pearson", "user_based": True},
        {"name": "cosine", "user_based": True},
        {"name": "msd", "user_based": True},
        {"name": "pearson_baseline", "user_based": True},
    ],
    'verbose': [False],
}

grid_search = GridSearch(
    KNNWithMeans,
    parameters_grid,
    test_measures,
    train_measures,
    max_workers=16,
)

best, ordering, raw = grid_search.fit(folds_without_index)

Completed 128/128 | Avg. time/task: 0m 1.2s | Time left: 0m 0.0s | Estimated completion time: 16:09:567
All tasks completed.


In [6]:
import json

with open('UBKNN_grid_best.results', 'w') as file_object:
    json.dump(best, file_object, indent=4)

with open('UBKNN_grid_ordering.results', 'w') as file_object:
    json.dump(ordering, file_object, indent=4)

with open('UBKNN_grid_raw.results', 'w') as file_object:
    json.dump(raw, file_object, indent=4)
